In [1]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from skimage import io
import os

# Define the model
$$\dot{A}(t) = r A(t) \left(1 - \frac{A(t)}{k} \right)$$

$$A(t) = \frac{k}{1 + (\frac{k}{A_0}-1) e^{-r t}}$$ 

A: Area

t: Time

r: Growth rate

k: Carrying capacity

In [2]:
# Define the logistic growth model
def area_logistic(t, a0, r, k):
    """
    t  :time
    a0 : area at time zer0
    r  : growth rate
    k  : carrying capacity
    """
    return k / (1 + (k/a0 - 1)*np.exp(-r*t))

def der1_area_logistic(t, a0, r, k):
    """
    t  :time
    a0 : area at time zer0
    r  : growth rate
    k  : carrying capacity
    """
    A = area_logistic(t, a0, r, k)
    return r*A*(1-A/k)

def der2_area_logistic(t, a0, r, k):
    """
    t  :time
    a0 : area at time zer0
    r  : growth rate
    k  : carrying capacity
    """
    A = area_logistic(t, a0, r, k)
    return (r**2)*A*(1-A/k)*(1-2*A/k)

# Make a preview of each movie beside its curves

In [3]:
# Read the area curves table
df = pd.read_pickle('Results/manipulated_area_curves.pkl')

# Make a preview of each movie 
for i in range(df.shape[0]):
#for i in [2]:
    # Read the time and area
    t, A = df['Time'][i], df['Area'][i]

    # Create an array of weights
    l = t.shape[0]
    sigmas = np.ones(l) * (np.max(A)/100)

    sigmas[:l//5] = sigmas[:l//5] * 0.1
    sigmas[l-2*l//5:] = sigmas[l-2*l//5:] * 0.1
    sigmas[l//5:l-2*l//5] = sigmas[l//5:l-2*l//5] * 3

    # Fit a general tanh function to the curves 
    popt_logistic, _ = curve_fit(area_logistic, t, A, sigma=sigmas,
                                 p0=(1, 0.01, np.max(A)),
                                 bounds=([0.1, 0.0001, np.min(A)], [np.min(A), 1, 2*np.max(A)]))

    # Read the corresponding movie according to it's protein name
    if df['Protein'][i] == 'Mouse Ecad':
        movie = io.imread(f'../RICM-Vesicles/{df["Date"][i]}_Ecad_dynamics/data/{df["Name"][i]}.tif')

    else:
        # Get the right directory name using the date
        for directory in sorted(os.listdir('../RICM-vesicles2/Data/')):
            if df['Date'][i] in directory:
                experiment = directory
        movie = io.imread(f'../RICM-vesicles2/Data/{experiment}/data/{df["Name"][i]}.tif')

    # Determine the period between each image
    period = movie.shape[0] // (3*6)

    # determine the frame rate
    dt = t[1] - t[0]

    # Define the figure size canvas
    fig, axs = plt.subplots(ncols=5, nrows=6, figsize=(20, 24), tight_layout=True)

    # Global title
    fig.suptitle(f'{i}: {df["Date"][i]} {df["Protein"][i]} {df["Discription"][i]} ({df["Name"][i]})', y=1, fontsize=25)

    # Specify the location of the big subplot 
    gs0 = axs[0, 0].get_gridspec()
    gs1 = axs[0, 2].get_gridspec()
    gs2 = axs[0, 4].get_gridspec()

    # Add the big subplots
    axbig1 = fig.add_subplot(gs0[0:2, 0:2])
    axbig2 = fig.add_subplot(gs1[2:4, 0:2])
    axbig3 = fig.add_subplot(gs2[4:6, 0:2])

    # Remove the underlying axes
    for l in range(6):
        for ax in axs[l,0:2]:
            ax.remove()

    # images of the movie 
    j = 0
    for k in range(6):
        for ax in axs[k, 2:6]:
            ax.imshow(movie[j*period], cmap='gray')
            if i in [2,3,4]:
                ax.set_title(f'{t[j*period]:.2f} s', fontsize='xx-large')
            else:
                ax.set_title(f'{j*period*dt:.2f} s', fontsize='xx-large')
            ax.set_xticks([])
            ax.set_yticks([])
            j += 1


    # Plot the data and it's logistic fitting
    axbig1.plot(t, A, label='Data', alpha = 0.6)                                                   # data
    axbig1.plot(t, area_logistic(t, *popt_logistic), label='Logistic', alpha = 0.6, color= 'g')    # logistic  
    axbig1.set_title(f'r = {popt_logistic[1]:.5f},    k = {popt_logistic[2]:.2f},    A0 = {popt_logistic[0]:.2f}', fontsize='xx-large')
    axbig1.set_ylabel('Area [ $\mu m^2$ ]', fontsize='xx-large')
    axbig1.tick_params(axis='both', which='major', labelsize=15)
    axbig1.grid(color = 'gray', alpha = 0.1)
    axbig1.legend(loc='lower right', fontsize='xx-large')
    
    # Plot line of best fit for the linear regime
    m, c = np.polyfit(t[:50], A[:50], 1)
    axsmall1 = fig.add_axes([.16, .69, .1, .1])  # location of the small gragh
    axsmall1.plot(t[:60], A[:60], label='Data', alpha = 0.6)                           # data
    axsmall1.plot(t[:60], m*t[:60]+c, label='Strait line', alpha = 0.6, color= 'r')    # straight line 
    axsmall1.set_title(f'm = {m:.5f}\nc = {c:.2f}', fontsize='x-large')
    axsmall1.grid(color = 'gray', alpha = 0.1)

    # Plot the velocity of the logistic fitting
    axbig2.plot(t, der1_area_logistic(t, *popt_logistic), label='Logistic Velocity', alpha = 0.6, color= 'g')   
    axbig2.set_ylabel('$dA/dt$ [ $\mu m^2 / S$ ]', fontsize='xx-large')
    axbig2.tick_params(axis='both', which='major', labelsize=15)
    axbig2.grid(color = 'gray', alpha = 0.1)
    axbig2.legend(loc='upper right', fontsize='xx-large')

    # Plot the acceleration of the logistic fitting
    axbig3.plot(t, der2_area_logistic(t, *popt_logistic), label='Logistic Acceleration', alpha = 0.6, color= 'g')   
    axbig3.set_xlabel('Time [ S ]', fontsize='xx-large')
    axbig3.set_ylabel('$d^2A/dt^2$ [ $\mu m^2 / S^2$ ]', fontsize='xx-large')
    axbig3.tick_params(axis='both', which='major', labelsize=15)
    axbig3.grid(color = 'gray', alpha = 0.1)
    axbig3.legend(loc='upper right', fontsize='xx-large')
    
    # Save the figure
    plt.savefig(f'Results/curves_with_initial_fitting_movie_preview/{i}:_{df["Date"][i]}_{df["Name"][i]}.png')
    
    # Close the plot
    plt.close()
    # plt.show()

/tmp/ipykernel_13497/2421331430.py:108: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig(f'Results/curves_with_initial_fitting_movie_preview/{i}:_{df["Date"][i]}_{df["Name"][i]}.png')


In [4]:
# Read the area curves table
df = pd.read_pickle('Results/manipulated_area_curves.pkl')

# Make a preview of each movie 
for i in range(df.shape[0]):
#for i in [2]:
    # Read the time and area
    t, A = df['Time'][i], df['Area'][i]

    # Create an array of weights
    l = t.shape[0]
    sigmas = np.ones(l) * (np.max(A)/100)

    sigmas[:l//5] = sigmas[:l//5] * 0.1
    sigmas[l-2*l//5:] = sigmas[l-2*l//5:] * 0.1
    sigmas[l//5:l-2*l//5] = sigmas[l//5:l-2*l//5] * 3

    # Fit a general tanh function to the curves 
    popt_logistic, _ = curve_fit(area_logistic, t, A, sigma=sigmas,
                                 p0=(1, 0.01, np.max(A)),
                                 bounds=([0.1, 0.0001, np.min(A)], [np.min(A), 1, 2*np.max(A)]))

    # Read the corresponding movie according to it's protein name
    if df['Protein'][i] == 'Mouse Ecad':
        movie = io.imread(f'../RICM-Vesicles/{df["Date"][i]}_Ecad_dynamics/data/{df["Name"][i]}.tif')

    else:
        # Get the right directory name using the date
        for directory in sorted(os.listdir('../RICM-vesicles2/Data/')):
            if df['Date'][i] in directory:
                experiment = directory
        movie = io.imread(f'../RICM-vesicles2/Data/{experiment}/data/{df["Name"][i]}.tif')

    # Determine the period between each image
    period = movie.shape[0] // (3*6)

    # determine the frame rate
    dt = t[1] - t[0]

    # Define the figure size canvas
    fig, axs = plt.subplots(ncols=5, nrows=6, figsize=(20, 24), tight_layout=True)

    # Global title
    fig.suptitle(f'{i}: {df["Date"][i]} {df["Protein"][i]} {df["Discription"][i]} ({df["Name"][i]})', y=1, fontsize=25)

    # Specify the location of the big subplot 
    gs0 = axs[0, 0].get_gridspec()
    gs1 = axs[0, 2].get_gridspec()
    gs2 = axs[0, 4].get_gridspec()

    # Add the big subplots
    axbig1 = fig.add_subplot(gs0[0:2, 0:2])
    axbig2 = fig.add_subplot(gs1[2:4, 0:2])
    axbig3 = fig.add_subplot(gs2[4:6, 0:2])

    # Remove the underlying axes
    for l in range(6):
        for ax in axs[l,0:2]:
            ax.remove()

    # images of the movie 
    j = 0
    for k in range(6):
        for ax in axs[k, 2:6]:
            ax.imshow(movie[j*period], cmap='gray')
            if i in [2,3,4]:
                ax.set_title(f'{t[j*period]:.2f} s', fontsize='xx-large')
            else:
                ax.set_title(f'{j*period*dt:.2f} s', fontsize='xx-large')
            ax.set_xticks([])
            ax.set_yticks([])
            j += 1


    # Plot the data and it's logistic fitting
    axbig1.plot(t, A, label='Data', alpha = 0.6)                                                           # data
    axbig1.plot(t, area_logistic(t, *popt_logistic), label='Logistic Fitting', alpha = 0.6, color= 'g')    # logistic    
    axbig1.set_title(f'r = {popt_logistic[1]:.5f},    k = {popt_logistic[2]:.2f},    A0 = {popt_logistic[0]:.2f}', fontsize='xx-large')
    axbig1.set_ylabel('Area [ $\mu m^2$ ]', fontsize='xx-large')
    axbig1.tick_params(axis='both', which='major', labelsize=15)
    axbig1.grid(color = 'gray', alpha = 0.1)
    axbig1.legend(loc='lower right', fontsize='xx-large')

    # Plot the velocity of the logistic fitting
    axbig2.plot(t, der1_area_logistic(t, *popt_logistic), label='Logistic Velocity', alpha = 0.6, color= 'g')   
    axbig2.set_ylabel('$dA/dt$ [ $\mu m^2 / S$ ]', fontsize='xx-large')
    axbig2.tick_params(axis='both', which='major', labelsize=15)
    axbig2.grid(color = 'gray', alpha = 0.1)
    axbig2.legend(loc='upper right', fontsize='xx-large')

    # Plot the acceleration of the logistic fitting
    axbig3.plot(t, der2_area_logistic(t, *popt_logistic), label='Logistic Acceleration', alpha = 0.6, color= 'g')   
    axbig3.set_xlabel('Time [ S ]', fontsize='xx-large')
    axbig3.set_ylabel('$d^2A/dt^2$ [ $\mu m^2 / S^2$ ]', fontsize='xx-large')
    axbig3.tick_params(axis='both', which='major', labelsize=15)
    axbig3.grid(color = 'gray', alpha = 0.1)
    axbig3.legend(loc='upper right', fontsize='xx-large')
    
    # Save the figure
    plt.savefig(f'Results/curves_with_derivatives_movie_preview/{i}:_{df["Date"][i]}_{df["Name"][i]}.png')
    
    # Close the plot
    plt.close()
    #plt.show()

In [10]:
# Read the area curves table
df = pd.read_pickle('Results/manipulated_area_curves.pkl')

# Make a preview of each movie 
for i in range(df.shape[0]):
# for i in [2]:
    # Read the time and area
    t, A = df['Time'][i], df['Area'][i]

    # Create an array of weights
    l = t.shape[0]
    sigmas = np.ones(l) * (np.max(A)/100)

    sigmas[:l//5] = sigmas[:l//5] * 0.1
    sigmas[l-2*l//5:] = sigmas[l-2*l//5:] * 0.1
    sigmas[l//5:l-2*l//5] = sigmas[l//5:l-2*l//5] * 3

    # Fit a general tanh function to the curves 
    popt_logistic, _ = curve_fit(area_logistic, t, A, sigma=sigmas,
                                 p0=(1, 0.01, np.max(A)),
                                 bounds=([0.1, 0.0001, np.min(A)], [np.min(A), 1, 2*np.max(A)]))

    # Read the corresponding movie according to it's protein name
    if df['Protein'][i] == 'Mouse Ecad':
        movie = io.imread(f'../RICM-Vesicles/{df["Date"][i]}_Ecad_dynamics/data/{df["Name"][i]}.tif')

    else:
        # Get the right directory name using the date
        for directory in sorted(os.listdir('../RICM-vesicles2/Data/')):
            if df['Date'][i] in directory:
                experiment = directory
        movie = io.imread(f'../RICM-vesicles2/Data/{experiment}/data/{df["Name"][i]}.tif')

    # Determine the period between each image
    period = movie.shape[0] // (3*4+2*2)

    # determine the frame rate
    dt = t[1] - t[0]

    # Define the figure size canvas
    fig, axs = plt.subplots(ncols=5, nrows=4, figsize=(20, 16), tight_layout=True)

    # Global title
    fig.suptitle(f'{i}: {df["Date"][i]} {df["Protein"][i]} {df["Discription"][i]} ({df["Name"][i]})', y=1, fontsize=25)

    # Specify the location of the big subplot 
    gs = axs[0, 0].get_gridspec()

    # Add the big subplots
    axbig = fig.add_subplot(gs[0:2, 0:2])

    # Remove the underlying axes
    for l in range(2):
        for ax in axs[l,0:2]:
            ax.remove()

    # images of the movie 
    j = 0
    for k in range(4):
        if k < 2:
            for ax in axs[k, 2:6]:
                ax.imshow(movie[j*period], cmap='gray')
                if i in [2,3,4]:
                    ax.set_title(f'{t[j*period]:.2f} s', fontsize='xx-large')
                else:
                    ax.set_title(f'{j*period*dt:.2f} s', fontsize='xx-large')
                ax.set_xticks([])
                ax.set_yticks([])
                j += 1
        else:
            for ax in axs[k, :]:
                ax.imshow(movie[j*period], cmap='gray')
                ax.set_title(f'{j*period*dt:.2f} s', fontsize='xx-large')
                ax.set_xticks([])
                ax.set_yticks([])
                j += 1


    # Plot the data and it's logistic fitting
    axbig.plot(t, A, label='Data', alpha = 0.6)                                                           # data
    axbig.plot(t, area_logistic(t, *popt_logistic), label='Logistic', alpha = 0.6, color= 'g')    # logistic
    axbig.set_title(f'r = {popt_logistic[1]:.5f},    k = {popt_logistic[2]:.2f},    A0 = {popt_logistic[0]:.2f}', fontsize='xx-large')
    axbig.set_ylabel('Area [ $\mu m^2$ ]', fontsize='xx-large')
    axbig.tick_params(axis='both', which='major', labelsize=15)
    axbig.grid(color = 'gray', alpha = 0.1)
    axbig.legend(loc='lower right', fontsize='xx-large')
    
    # Plot line of best fit for the linear regime
    m, c = np.polyfit(t[:50], A[:50], 1)
    axsmall1 = fig.add_axes([.15, .54, .1, .15])  # location of the small gragh
    axsmall1.plot(t[:60], A[:60], label='Data', alpha = 0.6)                           # data
    axsmall1.plot(t[:60], m*t[:60]+c, label='Strait line', alpha = 0.6, color= 'r')    # straight line 
    axsmall1.set_title(f'm = {m:.5f}\nc = {c:.2f}', fontsize='x-large')
    axsmall1.grid(color = 'gray', alpha = 0.1)
    
    # Save the figure
    plt.savefig(f'Results/curves_movie_preview/{i}:_{df["Date"][i]}_{df["Name"][i]}.png')
    
    # Close the plot
    plt.close()

/tmp/ipykernel_6149/3094463183.py:97: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig(f'Results/curves_movie_preview/{i}:_{df["Date"][i]}_{df["Name"][i]}.png')
